<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%bash
# apt-get update
# apt-get install g++ openjdk-8-jdk python-dev python3-dev
# pip3 install JPype1
# pip3 install konlpy

In [2]:
# %env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
# # Mecab 설치
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [4]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline
# 한글 지원 폰트
plt.rc('font', family='NanumBarunGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Mecab

vocab_size = 32000

data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_1000.csv')

In [6]:
data

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화
...,...,...,...,...,...
4908,3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
4909,3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
4910,3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
4911,3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [7]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면']

In [8]:
mecab = Mecab() 

data['tokenized'] = data['non_label_sentence'].apply(mecab.morphs)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
data

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class,tokenized
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화,"[아오, 요즘, 맨날, 늦, 자, 일찍, 일어나, 니까, 피부, 꺼슬꺼슬, 다크, ..."
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화,"[고양이, 키우, 건, 정말, 즐거운, 일, 야, 감자, 캐, 자, 우리, 집, 고..."
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화,"[조금, 전, 뉴스, 본, 거, 있, 어, 아니, 뉴스, 무슨, 기사, 떴, 어, ..."
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화,"[난, 강아지, 같, 푸근, 인상, 좋, 아, 오, 나, 원래, 고양이, 상, 좋,..."
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화,"[니, 폰, 자주, 편이, 가끔, 삘, 받, 으면, 긴, 는데, 거의, 안, 편, ..."
...,...,...,...,...,...,...
4908,3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화,"[준, 야, 넌, 대가리, 왜, 이렇게, 크, 냐, 내, 머리, 뭐, 밥, 먹, 으..."
4909,3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화,"[내, 지금, 너, 아들, 김길준, 데리, 있, 어, 살리, 싶, 으면, 계좌, 1..."
4910,3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화,"[나, 씨, 같, 사람, 보, 참, 신기, 더라, 어떻게, 저렇게, 살, 왜, 그래..."
4911,3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화,"[누구, 맘대로, 여기, 서, 장사, 래, 이게, 무슨, 일, 입니까, 남, 구역,..."


In [9]:
conversation = data['tokenized'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(conversation)

In [10]:
len(tokenizer.word_index)

17536

In [11]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 17536
등장 빈도가 1번 이하인 희귀 단어의 수: 6836
단어 집합에서 희귀 단어의 비율: 38.98266423357664
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.470831872040209


In [12]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 10702


In [13]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(conversation)
X_data = tokenizer.texts_to_sequences(conversation)

In [14]:
padded = pad_sequences(X_data)
print(padded)

[[   0    0    0 ...  451    7  433]
 [   0    0    0 ...  223  530    7]
 [   0    0    0 ...    1 1441    1]
 ...
 [   0    0    0 ...    3   13  193]
 [   0    0    0 ...  207   11   17]
 [   0    0    0 ...  419   57 2098]]


In [15]:
np.shape(padded)

(4913, 485)

In [16]:
train_data = padded
train_label = data['class']

In [17]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화': 4}

In [18]:
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)
print(train_label)

      0  1  2  3  4
0     0  0  0  0  1
1     0  0  0  0  1
2     0  0  0  0  1
3     0  0  0  0  1
4     0  0  0  0  1
...  .. .. .. .. ..
4908  0  0  0  1  0
4909  0  1  0  0  0
4910  0  0  1  0  0
4911  0  1  0  0  0
4912  0  0  1  0  0

[4913 rows x 5 columns]


In [19]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3930 491 492
3930 491 492


In [20]:
embedding_dim = 256
hidden_units = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout = 0.3)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout = 0.3)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout = 0.3)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2, dropout = 0.3)))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         5479424   
                                                                 
 bidirectional (Bidirectiona  (None, None, 512)        1574912   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 512)        1574912   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 512)        1574912   
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              656384    
 nal)                                                   

In [21]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint(filepath='/content/sample_data/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=64,
                    callbacks=[es, mc],
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
62/62 [==============================] - ETA: 0s - loss: 1.1003 - accuracy: 0.4903
Epoch 1: val_accuracy improved from -inf to 0.73320, saving model to /content/sample_data/best_model.h5
62/62 [==============================] - 47s 558ms/step - loss: 1.1003 - accuracy: 0.4903 - val_loss: 0.6505 - val_accuracy: 0.7332
Epoch 2/10
62/62 [==============================] - ETA: 0s - loss: 0.5475 - accuracy: 0.7893
Epoch 2: val_accuracy improved from 0.73320 to 0.81874, saving model to /content/sample_data/best_model.h5
62/62 [==============================] - 31s 493ms/step - loss: 0.5475 - accuracy: 0.7893 - val_loss: 0.5154 - val_accuracy: 0.8187
Epoch 3/10
62/62 [==============================] - ETA: 0s - loss: 0.3121 - accuracy: 0.8992
Epoch 3: val_accuracy did not improve from 0.81874
62/62 [==============================] - 30s 492ms/step - loss: 0.3121 - accuracy: 0.8992 - val_loss: 0.6017 - val_accuracy: 0.8187
Epoch 4/10
62/62 [==============================] - ETA: 0s 

In [22]:
loaded_model = load_model('/content/sample_data/best_model.h5')
pred = loaded_model.predict(test_X)
print("테스트 정확도: %.4f" % (loaded_model.evaluate(test_X, test_Y)[1]))

16/16 [==============================] - 4s 104ms/step - loss: 0.6303 - accuracy: 0.7947
테스트 정확도: 0.7947


In [23]:
from sklearn.metrics import classification_report

pred_label = []
test_label = []
test_index = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))
  test_index.append(test_Y.index[i])

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.88      0.73      0.80        94
           2       0.82      0.75      0.79       110
           3       0.64      0.80      0.71       115
           4       0.92      0.94      0.93       104

    accuracy                           0.79       492
   macro avg       0.81      0.79      0.79       492
weighted avg       0.81      0.79      0.80       492



In [24]:
test_X

array([[   0,    0,    0, ...,   61,   50,  351],
       [   0,    0,    0, ...,    8,   11,   17],
       [   0,    0,    0, ...,   23,   11,   17],
       ...,
       [   0,    0,    0, ..., 1457,  144,    4],
       [   0,    0,    0, ...,  299,   52,  117],
       [   0,    0,    0, ...,  258, 1689,  258]], dtype=int32)

In [25]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화': 4}
lidx2labels = {str(v) : k for k, v in labels.items()}

In [26]:
test_index
# 잘못 분류한 부분 출력

error_idx = []
error_label = []
error_data_idx = []

for idx, val in enumerate(pred_label):
  if val != test_label[idx]:
    error_data_idx.append(test_index[idx])
    error_idx.append(idx)
    error_label.append(val)

In [27]:
len(error_idx)

101

In [28]:
error = test_X[error_idx]

In [29]:
error_text = tokenizer.sequences_to_texts(error)

In [30]:
result = []

for i in range(len(error_idx)):
  result.append([data['non_label_sentence'][error_data_idx[i]], lidx2labels[error_label[i]], data['class'][error_data_idx[i]]])

result

[['너 이거 이렇게 하라 했어 안 했어\n네  저번에 선배님이 설명해주신 대로 했는데요\n장난해  내가 언제 이렇게 하라 했어 너 머리는 장식이니\n제가 착각했나 봐요 죄송합니다\n죄송하면 다야  정말 답답해 죽겠다 죽겠어\n죄송합니다\n죄송할 짓을 하지 말아야지 너 오늘 이거 다하기 전까지 집 못 가\n죄송해요\n좌속하다는 말밖에 못 하지 너는\n앞으로 실수 안 하겠습니다',
  '기타 괴롭힘 대화',
  '직장 내 괴롭힘 대화'],
 ['아줌마 손에 든 거 뭐예요\n장바구니인데요\n아니 장바구니 인건 아는데 뭐 들었어요\n오늘 애들 먹일 소고 기요\n아줌마 미안한데 나도 집에 애가 둘이나 있는데 돈이 없어요\n네\n소고기 좀 같이 나눠먹게요 아줌마는 옷도 잘 입은 거 보니 있어 보이는데 나는 신랑이 우리 버리고 가서 연락도 안 돼서 돈도 없어요\n주민센터 같은데 연락해서 도움 좀 받지 그래요\n아 주말에 주민센터 여는 거 봤어요  우선 당장 굶어죽게 생겼으니까 좀 주라고요\n죄송해요 저 가볼게요\n아줌마 그냥 가면 나 아줌마 집까지 쫓아가서 그 집 앞에 앉아서 애들 기다려요\n기다려서 뭐 하게요\n글쎄 뭐 할까  궁금하면 그냥 가시든가 아니면 그냥 소고기 좀 주고 가던가',
  '기타 괴롭힘 대화',
  '갈취 대화'],
 ['뭐라고  네가 뭔데\n뭐가요\n너 나 누군지 몰라\n몰라요\n그래  너 잘 만났다 이리 와봐\n제가 왜요\n그냥 얼굴이 맘에 안 들어\n네\n하 오늘 기분도 안 좋은데 너 잘 걸렸다\n왜 그러세요',
  '기타 괴롭힘 대화',
  '협박 대화'],
 ['아오 어디서 썩은 냄새가 나냐 야 창문 열어 구토 나오겠네\n그러네요 자꾸 어디서 썩은 냄새가 나는지 나 원 참\n야 너 안 씻고 왔냐  너한테 하수구 썩는 냄새나 야 얘 냄새 맡아봐 무슨 시체 썩고 있냐\n킁킁 아오 씨 야 너 씻고 다녀 집에서 물이 안 나와  토악질 나오겠네\n네   저 오늘 아침에 샤워하고 왔는데요\n그러면 그냥 어릴 때부터 나서 몸에 배었나 봐 냄새가 아니면 

In [31]:
result_df = pd.DataFrame(result, columns=['text', 'predict', 'true'])

In [32]:
result_df

,text,predict,true
0,너 이거 이렇게 하라 했어 안 했어\n네 저번에 선배님이 설명해주신 대로 했는데요...,기타 괴롭힘 대화,직장 내 괴롭힘 대화
1,아줌마 손에 든 거 뭐예요\n장바구니인데요\n아니 장바구니 인건 아는데 뭐 들었어요...,기타 괴롭힘 대화,갈취 대화
2,뭐라고 네가 뭔데\n뭐가요\n너 나 누군지 몰라\n몰라요\n그래 너 잘 만났다 ...,기타 괴롭힘 대화,협박 대화
3,아오 어디서 썩은 냄새가 나냐 야 창문 열어 구토 나오겠네\n그러네요 자꾸 어디서 ...,기타 괴롭힘 대화,직장 내 괴롭힘 대화
4,야 너네 집에 키위 농장한다며\n어 응\n내가 키위 진짜 좋아하는데 좀 얻어먹어보자...,기타 괴롭힘 대화,갈취 대화
...,...,...,...
96,오빠야는 음식 잘하낭\n아니 요리를 안 하지\n요리 해본 적은 있어\n계란후라이랑 ...,직장 내 괴롭힘 대화,일반 대화
97,내 사진 몰래 찍은 거 사진 올라왔던데 네가 찍은 거니\n나 아니야\n우리 관계한 ...,기타 괴롭힘 대화,협박 대화
98,와 이거 색연필 딱 봐도 비싸 보인다\n아 아빠가 수행평가 있다고 하니까 사주셨어\...,기타 괴롭힘 대화,갈취 대화
99,안녕하세요\n네\n혹시 이나연 씨인가요\n네 맞아요\n왜 그러시죠\n돈을 안 갚으셨...,갈취 대화,협박 대화


In [33]:
# True : 협박 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='협박 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
동생아  너 내가 어제 설거지 해놓으라고 했지
형 나 어제오늘 바쁘다고 말했잖아
어쭈 그래서 안 하고 버틴다고
이번만 형이 해주라
설거지 안 해두면 너 지난번에 성적표 숨긴 거 엄마께 말한다
아  그건 좀 아니지
그리고 저번에 오티 간다고 거짓말하고 클럽에서 밤새운 것도 다 말할 거야
형 진짜이러 가야
싫으면 해놓던가
아  하면 되잖아  설거지

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  협박 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
내 사진 몰래 찍은 거 사진 올라왔던데 네가 찍은 거니
나 아니야
우리 관계한 사진 찍을 사람 너밖에 없잖아
나 아니라니까 자꾸 그러네 누군가 몰카 설치했나 보지
핸드폰 줘봐
아이 씨  나를 못 믿어 사람을 못 믿고 지랄이야
사실 확인을 해보려는 거뿐인데 왜 화를 내
나 아니라니까 네가 자꾸 네가 의심하잖아 짜증 나네
확인해보자고
짜증 나게 하지 마 죽여버리기 전에

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  협박 대화


In [34]:
# True : 갈취 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='갈취 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
야 김수미 시간 있냐
무슨 일이야
잠깐 와봐
왜 그래
너 그 패딩 이번 신상이지 이쁘다
어어 생일 선물이야
나 오늘 중요한 날인데 좀 빌리자
안되는데 나도 입어야 되는데
잠깐 입고 줄게 그 대신 내 거 입어
아 안되는데

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  갈취 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
내가 나 만날 때 돈 가지고 오라고 했잖아
미안해
에이  그럼 내가 나쁜 사람처럼 보이지
오늘은 진짜 없어서 그래
그럼 없는 만큼 몸으로 때우면 되겠네
미안해  이제 나한테 그만 좀 해
그만해   아니 내가 어려운 부탁했어  그냥 하루에 조금씩만 가지고 오라는데 그게 안될까
그냥 나 좀 내버려 둬
네가 진짜 맞아야 정신이 들겠지  어디부터 맞을래
아  미안해 여기 있어
그래  이제야 정신이 들지
이게 진짜 끝이었으면

예측 라벨 :  협박 대화

실제 라벨 :  갈취 대화


In [35]:
# True : 직장 내 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='직장 내 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
박하선 일병 빠졌네
일병 박하선
야 불렀는데 대답 안 하냐
죄송합니다
네 맞선임 누구냐   네 위로 내 아래로 다 집합
죄죄죄죄 죄송합니다  그  것만은
야 이 지발 놈아 선임 말이 우습냐
아아  아닙니다
알아먹었으면 빨리 안 움직여
네 알겠습니다

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  직장 내 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
사장님 안녕하세요 출근하셨습니까 생신 축하합니다   제가 편지도 써왔습니다
오 편지  야 감동이다  한 번 읽어볼게  음  야 근데 너 글씨체가 왜 이러냐
아 제가 어릴 때부터 글씨가 지렁이 같다는 말은 많이 들었습니다
정성이 안 들어간 거 아니야  우리 5살 딸이 글씨 써도 너보다는 잘 쓸 거 같아
에이 부장님 왜 이러세요 그냥 마음만 봐주세요 글의 내용  제가 거의 3일을 고민하면서 쓴 편지라고요
글의 내용이 눈에 안 들어와  글씨가 너무 이상해서  야 이 정도면 너네 어머니 애 낳고 미역국 안 먹은 거 아니야  네가 그래서 모질라 진 거지
사장님  사장님 생신이라고 제가 선물이랑 편지 하나하나 다 준비한 건데 너무하세요
야 왜 아니 딱 글씨체만 봐도 알 수 있는 게 약간 글씨가 불안정한 게 너네 집안도 정말 불안정했던 거 아니냐
아닙니다  자꾸 그렇게 얘기하실 거면 돌려주세요  저도 이런 말 듣고 드리고 싶은 생각 없어요
야 그래 가져가 가져가 너네 부모도 이렇게 졸렬하지   졸장부 밑에서 나온 애가 뭐 이렇지  너 글씨체만 봐도 사회생활 되게 못 할거 같은 글씨체야 알아

예측 라벨 

In [36]:
# True : 기타 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='기타 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
야 저 사람 봐봐 진짜 그 티브이에 나오는 못생긴 개그맨 닮지 않았냐
푸하하하핳 야 진짜 못생겼다 헐 너무 크게 얘기했나 봐
저기요 지금 저한테 말씀하신 거예요
아닌데요
아 그래요
네 그냥 가던 길 가시면 될 거 같은데요
야 저 사람 못생긴 대신 귀는 밝은가 봐
자기가 못생긴지 아나 보네
저기요 저한테 하신 말 맞네요 저 지금 심각한 모욕감을 느끼고 있는데 빨리 사과하세요
네  다른 사람 보고 한말인데요  그쪽이 너무 예민한 거 같은데요
제가 다 들었어요 그 두 분들도 잘생기신 편은 아닌 거 같은데 그런 말 할 권리가 어디 있나요 대체

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  기타 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
아 정글 씨발새끼 정글에 엄마 묻어놨냐  계속 찾아다니네
그니까 쉬발
지가 갱 당해 뒤져놓고 지랄하니
엄마 그만 찾아 벌레 년아 내가 넥서스에 묻어놨음 저거 터트려야 찾을 수 있어
어휴 버러지 년 수준 보인다
정글 씨발련아 뭐 하냐  채팅 칠 시간에 갱이나 쳐 다녀  상대 정글 10번 왔어 미친년아
쌍으로 똥 싸놓고 지랄하네
정글 차이
정글 차이
정글 차이

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  기타 괴롭힘 대화


In [37]:
# True : 일반 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='일반 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
넌 예전에 직장생활 오래 했었어
응 졸업하고 결혼 전까지는 계속 일 다녔지
오호 그럼 직장생활 하면서 많이 힘든 건 없었고
육아하니 갑자기 또 직장생활이 그립네
원래 다들 그런가 봐
내 친구도 그러더라
직장생활보다 육아가 더 힘든 거 같아
다시 일 하고 싶네
진짜 제발 업무를 시켜달라고 하던데 육아 말고
일 안 하지 너무 오래돼서 다시 할 수 있을지 의문이야
하루 종일 아기 보는 것보다 직장에서 가끔 수다도 떨고 하는 게 좋을지도
이게 지금 내 직업이야
육아도 일이지
경력이 아무래도 단절 되니까 그런 걱정도 되기 시작하겠네 넌
얼른 키워 놓고 복직하고 싶어
육아도 임금을 받고 해야 한다고 생각해 중노동
나중에 크면 보상 받아야겠다
울 아기는 이걸 알라나
알아줘야지
그럼 지금은 육아 휴직을 해놓은 상태야
아니 그냥 그만두고 나와서 다른 종목으로 일해볼까 싶어

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  일반 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
오빠야는 음식 잘하낭
아니 요리를 안 하지
요리 해본 적은 있어
계란후라이랑 김치 볶음밥 정도
오 그것만 하면 자취력 좋은 거 아니야
집에 계란이랑 김치밖에 없다
그래도 계란은 사놓네
라면이랑 계란 김치 필수템 아니가
맞다 김치도 사 먹낭
아니 김치는 본가에서 가져온다 가까워서
김치만 있으면 밥 굶지는 않을 듯
라면이랑 김치만 먹는 거 같은데
그게 어디야 라면에 김치 빠지면 안 돼
니도 먹을 줄 아네
라면 끓일 때 김치도 넣고 따로 먹기도 하고 꿀맛
참치 넣어도 졸맛탱

예측 라벨 :  직